In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom

from pymedphys_analysis.tpscompare import absolute_scans_from_mephysto
from pymedphys_dicom.dicom.dose import dicom_dose_interpolate

In [ ]:
ROOT_DIR = Path(r"S:\Physics\Monaco\Model vs Measurement Comparisons")

DICOM_DIR = next(ROOT_DIR.glob(r"Beam Models\External *\DICOM Dose Exports"))
MEASUREMENTS_DIR = ROOT_DIR.joinpath(r"Measurements\RCCC\Photons\With Flattening Filter")
RESULTS = ROOT_DIR.joinpath(r"Results\RCCC\external")

In [ ]:
mephysto_files = list(MEASUREMENTS_DIR.glob('06MV * Open.mcc'))
keys = [
    re.match('06MV (\d\dx\d\d) Open.mcc', filepath.name).group(1)
    for filepath in mephysto_files
]

mephysto_file_map = {
    key: filepath for key, filepath in zip(keys, mephysto_files)
}

mephysto_file_map

In [ ]:
dicom_file_map = {
    key: DICOM_DIR.joinpath(f'06MV_{key}.dcm')
    for key in keys
}

dicom_file_map

In [ ]:
dicom_dataset_map = {
    key: pydicom.read_file(str(dicom_file_map[key]), force=True)
    for key in keys
}

In [ ]:
absolute_dose_table = pd.read_csv(MEASUREMENTS_DIR.joinpath('AbsoluteDose.csv'), index_col=0)
absolute_dose = absolute_dose_table['d10 @ 90 SSD']['6 MV']
absolute_dose

In [ ]:
output_factors = pd.read_csv(MEASUREMENTS_DIR.joinpath('OutputFactors.csv'), index_col=0)
output_factors

In [ ]:
absolute_dose_per_field = {
    key: output_factors[key]['6 MV'] * absolute_dose
    for key in keys
}

absolute_dose_per_field

In [ ]:
absolute_scans_per_field = {
    key: absolute_scans_from_mephysto(
        mephysto_file_map[key], 
        absolute_dose_per_field[key], 100)
    for key in keys
}

# absolute_scans_per_field

In [ ]:
getter = operator.itemgetter('displacement', 'dose')

In [ ]:
for key in keys:
    plt.plot(*getter(absolute_scans_per_field[key]['depth_dose']))

In [ ]:
def plot_tps_meas_diff(displacement, meas_dose, tps_dose):
    diff = tps_dose - meas_dose
    diff_range = np.max(np.abs(diff))
    
    lines = []

    fig, ax1 = plt.subplots(figsize=(12,6))

    lines += ax1.plot(displacement, meas_dose, label='Measured Dose')
    lines += ax1.plot(displacement, tps_dose, label='TPS Dose')
    ax1.set_ylabel('Dose (Gy)')
    
    x_bounds = [np.min(displacement), np.max(displacement)]
    ax1.set_xlim(x_bounds)

    ax2 = ax1.twinx()

    lines += ax2.plot(displacement, diff, color='C3', alpha=0.5, label='Residuals [TPS - Meas]')
    ax2.plot(x_bounds, [0, 0], '--', color='C3', lw=0.5)
    ax2.set_ylim([-diff_range, diff_range])
    ax2.set_ylabel('Dose difference [TPS - Meas] (Gy)')

    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels)

#     fig.tight_layout()

In [ ]:
def plot_pdd_diff(key):
    depth, meas_dose = getter(absolute_scans_per_field[key]['depth_dose'])

    y = depth - 300
    tps_dose = np.squeeze(dicom_dose_interpolate(dicom_dataset_map[key], ([0], y, [0]))) / 10

    plot_tps_meas_diff(depth, meas_dose, tps_dose)
    plt.title(f'Depth Dose Comparisons | {key} field')
    

for key in keys:
    plot_pdd_diff(key)
    filename = RESULTS.joinpath(f'{key}_pdd.png')
    plt.savefig(filename)
    plt.show()

In [ ]:
def plot_profile_diff(key, depth, direction):
    displacement, meas_dose = getter(absolute_scans_per_field[key]['profiles'][depth][direction])
    y = [depth - 300]
    
    if direction is 'inplane':
        grid = (displacement, y, [0])
    elif direction is 'crossplane':
        grid = ([0], y, displacement)
    else:
        raise ValueError("Expected direction to be equal to 'inplane' or 'crossplane'")

    tps_dose = np.squeeze(dicom_dose_interpolate(dicom_dataset_map[key], grid)) / 10
    plot_tps_meas_diff(displacement, meas_dose, tps_dose)
    plt.title(f'{direction.capitalize()} Profile Comparisons | {key} field | Depth: {depth} mm')
    

for key in keys:
    depths = absolute_scans_per_field[key]['profiles'].keys()
    for depth in depths:
        for direction in ['inplane', 'crossplane']:
            plot_profile_diff(key, depth, direction)
            filename = RESULTS.joinpath(f'{key}_profile_{depth}mm_{direction}.png')
            plt.savefig(filename)
            plt.show()